In [1]:
import pandas as pd
import numpy as np
from QualtricsAPI.Setup import Credentials
from QualtricsAPI.Survey import Responses

In [2]:
pd.set_option('display.max_columns', None)

In [4]:
# versions

print('\n'.join(f'{m.__name__}=={m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))

pandas==1.4.3
numpy==1.22.3


# Import the data

In [45]:
# import data through API and anonymize

qtoken = '...' # personal token, can get this from qualtrics
qdc = '...' # country where the data is stored -- can find this in your qualtrics account
s_id = '...' # survey id, can find this in qualtrics account

Credentials().qualtrics_api_credentials(token=qtoken,data_center=qdc)
df = Responses().get_survey_responses(survey=s_id)

In [46]:
# store questions and drop survey information rows

questions = df.iloc[0]
df = df.iloc[2:]

In [47]:
# select only responses after survey was sent out to participants

df['start'] = pd.to_datetime(df['StartDate'])
df['end'] = pd.to_datetime(df['EndDate']) 
df = df.loc[(df['start'] > "2022-09-15") & (df['end'] < "2022-11-14")]

# column to indicate if a response is part of the registered dataset
df['main_data'] = np.where(df['end'] < "2022-10-16",1,0)

In [48]:
# export raw data for upload

# make a copy and add the question descriptions

df_raw = df.copy()
a = pd.Series(['startdate','enddate','main_data'])
questions = pd.concat([questions,a])
df_raw.iloc[0] = questions

# remove columns to safeguard anonimity: countries and comments removed

df_raw.drop(columns = ['D3','Q41'])

# save

df_raw.to_csv(r'...', sep = ';')


In [49]:
#rename columns

df = df.rename(columns = {'Duration (in seconds)': 'duration', 'D1': 'taxonomist?','D2':'professional','D3':'Country',
                         'D4': 'specialization', 'D5': 'broad_reader', 'D6': 'experience'})
df.duration = df.duration.astype('int')

In [50]:
# remove responses that took less than 180 secs (as preregistered)

print(f'number of responses that took less than 180 secs (and are dropped): {len(df.loc[(df.duration < 180)])}')
df = df.loc[(df.duration > 150)]

number of responses that took less than 180 secs (and are dropped): 106


In [51]:
df['taxonomist?'].value_counts()

1    456
2     97
Name: taxonomist?, dtype: int64

# Variables, categories etc we use

In [52]:
demo = ['taxonomist?', 'professional',
       'Country', 'specialization', 'broad_reader', 'experience', 'species_concepts']
agree = ['c1_agree', 'c2_agree','c3_agree']
accept = ['c1_accept','c2_accept','c3_accept']
cases = ['condition_c1','condition_c2','condition_c3']
demo2 = ['taxonomist?','professional','specialization','broad_reader','experience','species_concepts']

In [53]:
#create column for condition for case1

df['condition_c1'] =  np.where(
    df['1A_Agree'].notnull(), 'Neutral', np.where(
    df['1B_Agree'].notnull(), 'Threatened', np.where(
    df['1C_Agree'].notnull(), 'Abundant', 'No response')) )

#create shared columns for case 1 agree 

df['c1_agree'] = np.where(
    df['condition_c1'] == 'Neutral', df['1A_Agree'], np.where(
    df['condition_c1'] == 'Threatened', df['1B_Agree'], np.where(
    df['condition_c1'] == 'Abundant', df['1C_Agree'], 'No response')) )

#create shared columns for case 1 accept

df['c1_accept'] = np.where(
    df['condition_c1'] == 'Neutral', df['1A_Accept'], np.where(
    df['condition_c1'] == 'Threatened', df['1B_accept'], np.where(
    df['condition_c1'] == 'Abundant', df['1C_Accept'], 'No response')) )


#create column for condition for case2

df['condition_c2'] =  np.where(
    df['2A_Agree'].notnull(), 'Neutral',  np.where(
    df['2C_Agree'].notnull(), 'Morphology', np.where(
    df['2D_Agree'].notnull(), 'DNA', np.where(
    df['2E_Agree'].notnull(), 'Habitat', 'No response')))) 

#create shared columns for case 2 agree 

df['c2_agree'] =  np.where(
    df['condition_c2']== 'Neutral', df['2A_Agree'],  np.where(
    df['condition_c2']== 'Morphology', df['2C_Agree'], np.where(
    df['condition_c2']== 'DNA', df['2D_Agree'], np.where(
    df['condition_c2']== 'Habitat', df['2E_Agree'], 'No response')))) 

#create shared columns for case 2 accept

df['c2_accept'] =  np.where(
    df['condition_c2']== 'Neutral', df['2A_Accept'],  np.where(
    df['condition_c2']== 'Morphology', df['2C_Accept'], np.where(
    df['condition_c2']== 'DNA', df['2D_Accept'], np.where(
    df['condition_c2']== 'Habitat', df['2E_Accept'], 'No response')))) 


#create shared columns for case 2 lacking

df['c2_lack_morph'] =  np.where(
    df['condition_c2']== 'Neutral', df['2A_Lack_2'],   np.where(
    df['condition_c2']== 'DNA', df['2D_Lack_1'], np.where(
    df['condition_c2']== 'Habitat', df['2E_Lack_1'], 0)))

df['c2_lack_dna'] =  np.where(
    df['condition_c2']== 'Neutral', df['2A_Lack_3'],  np.where(
    df['condition_c2']== 'Morphology', df['2C_Lack_2'],  np.where(
    df['condition_c2']== 'Habitat', df['2E_Lack_2'],0)))

df['c2_lack_hab'] =  np.where(
    df['condition_c2']== 'Neutral', df['2A_Lack_4'],  np.where(
    df['condition_c2']== 'Morphology', df['2C_Lack_3'], np.where(
    df['condition_c2']== 'DNA', df['2D_Lack_3'], 0))) 

df['c2_lack_other'] =  np.where(
    df['condition_c2']== 'Neutral', df['2A_Lack_5_TEXT'],  np.where(
    df['condition_c2']== 'Morphology', df['2C_Lack_4_TEXT'], np.where(
    df['condition_c2']== 'DNA', df['2D_Lack_4_TEXT'], np.where(
    df['condition_c2']== 'Habitat', df['2E_Lack_4_TEXT'], 0))) )

#create column for condition for case3

df['condition_c3'] =  np.where(
    df['3A_Agree'].notnull(), 'Neutral', np.where(
    df['3B_Agree'].notnull(), 'No gene flow', np.where(
    df['3C_Agree'].notnull(), 'Gene flow', 'No response')) )

#create shared columns for case 3 agree 

df['c3_agree'] = np.where(
    df['condition_c3'] == 'Neutral', df['3A_Agree'], np.where(
    df['condition_c3'] == 'No gene flow', df['3B_Agree'], np.where(
    df['condition_c3'] == 'Gene flow', df['3C_Agree'], 'No response')) )

#create shared columns for case 3 accept

df['c3_accept'] = np.where(
    df['condition_c3'] == 'Neutral', df['3A_Accept'], np.where(
    df['condition_c3'] == 'No gene flow', df['3B_Accept'], np.where(
    df['condition_c3'] == 'Gene flow', df['3C_Accept'], 'No response')) )


In [54]:
# change '1 and '2' into 'yes' and 'no'

df[agree+accept] = df[agree+accept].replace({'1':'yes','2':'no'})

In [55]:
# get meaningful values for demo questions

df['professional'].replace({'1': 'pro','3':'amateur'}, inplace = True)
df['taxonomist?'].replace({'1': 'taxonomist','2':'non_taxonomist'}, inplace = True)
df['broad_reader'].replace({'1': 'never','2':'rarely','3':'once in a while','4':'regularly','5':'very often'}, inplace = True)
df['specialization'].replace({'1': 'Mammals','2':'Birds','3':'Reptiles and Amphibia','4':'Fish','5':'Non-vertebrate Deuterostomes',
                         '6':'Insects','7':'Non-insect Arthropods', '8':'Molluscs','9':'Remaining invertebrates',
                         '14':'Algae','10':'Protists (non-algae)','11':'Plants','12':'Fungi','13':'Prokaryotes'}, inplace = True)
df['experience'].replace({'1': '0-5','2':'6-10','3':'11-20','4':'21-30','5':'31+'}, inplace = True)

In [56]:
# turn experience into an ordered categorical

df.experience.fillna('No response')
cat_type = pd.CategoricalDtype( categories = ['0-5','6-10', '11-20', '21-30', '31+','No response' ], ordered=True)
df['experience'] = df['experience'].astype(cat_type)

In [57]:
#change values for the 'species concept' question into meaningful phrases

df['species_concepts'] = df['D7'].replace({'1':'BSC', '2':'ESC', '3':'GCSP', '4': 'PSCdia','5': 'PSCmono', '6': 'other'})

df['SC_full'] = np.where(
    df['species_concepts'].notnull(), df['species_concepts'], np.where(
    df['D7_6_TEXT'] == 'other', df['D7_6_TEXT'],  'No response'))

georgia = europe
azeirbejan = asia
armenia = asia
cyprus = europe
turkey = europe

In [58]:
# replace country codes by names, and add country classifications (continent, least developed, low income)
# note: country level-data will not be uploaded 

# list of countries from world bank
countries = pd.read_csv(r'...', sep = ';')
countries = countries.iloc[1:,:].copy()

countries.high_income_OECD.fillna(1, inplace = True)
countries.UN_least_developed.fillna(0, inplace = True)


countries.columns = ['country','code', 'low_income', 'least_developed','continent']
a = dict(zip(countries.code,countries.country))
b = dict(zip(countries.country, countries.continent))
c = dict(zip(countries.country, countries.least_developed))
d = dict(zip(countries.country, countries.low_income))

df.Country.fillna('No response', inplace = True)
df.Country = df.Country.replace(a)

df['continent'] = list(df.Country)

df['least_developed'] = df['continent'].replace(c)
df['low_income'] = df['continent'].replace(d)
df['continent'] = df['continent'].replace(b)

In [59]:
# drop columns that we don't need

df = df.drop(columns=['StartDate', 'EndDate','RecordedDate', 'Progress','ResponseId','DistributionChannel',
       'UserLanguage','1A_Agree',
       '1A_Accept', '1B_Agree', '1B_accept', '1C_Agree', '1C_Accept',
       '2A_Agree', '2A_Lack_2', '2A_Lack_3', '2A_Lack_4', '2A_Lack_5',
       '2A_Lack_5_TEXT', '2A_Accept', '2C_Agree', '2C_Lack_2', '2C_Lack_3',
       '2C_Lack_4', '2C_Lack_4_TEXT', '2C_Accept', '2D_Agree', '2D_Lack_1',
       '2D_Lack_3', '2D_Lack_4', '2D_Lack_4_TEXT', '2D_Accept', '2E_Agree',
       '2E_Lack_1', '2E_Lack_2', '2E_Lack_4', '2E_Lack_4_TEXT', '2E_Accept',
       '3A_Agree', '3A_Accept', '3B_Agree', '3B_Accept', '3C_Agree',
       '3C_Accept', 'D7', 'D7_6_TEXT'])

In [60]:
#dimensions
df.shape

(581, 32)

In [61]:
#check for duplicate rows

df.duplicated().value_counts()

False    581
dtype: int64

In [62]:
#save the df for use in other notebooks and for upload

df.to_csv(r'...', index=False)
